In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
import warnings

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1500)

warnings.filterwarnings('ignore')
np.random.seed(1)

/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [2]:
example = pd.read_csv('example.csv')

In [35]:
train_df, test_df = train_test_split(example, train_size = 0.8)
X_train = train_df.drop([ 'Grant.Status'], axis=1)
y_train = train_df['Grant.Status']
X_test = test_df.drop([ 'Grant.Status'], axis=1)
y_test = test_df['Grant.Status']

In [36]:
gbclf = GradientBoostingClassifier(n_estimators=20, max_depth=4, verbose=0)
tuned_parameter = [{'n_estimators':[20,30,40,50], 'max_depth':[5, 6, 7, 8], 'max_features':[0.2,0.3,0.4,0.5], 'subsample':[0.2,0.3,0.4,0.5]}]
gs_clf = GridSearchCV(gbclf, tuned_parameter, cv=5, scoring='roc_auc')
gs_clf.fit(X_train, y_train)
print('best parameters set found: ')
print(gs_clf.best_params_)

best parameters set found: 
{'max_depth': 7, 'max_features': 0.4, 'n_estimators': 50, 'subsample': 0.5}


In [37]:
gbclf = GradientBoostingClassifier(**gs_clf.best_params_)
gbclf.fit(X_train,y_train)
X_train_leaves = gbclf.apply(X_train)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbclf.fit(X_test,y_test)
X_test_leaves = gbclf.apply(X_test)[:,:,0]

In [45]:
X_train_arr=X_train_leaves.tolist()
X_test_arr=X_test_leaves.tolist()


[[6.0,
  21.0,
  30.0,
  130.0,
  46.0,
  14.0,
  39.0,
  101.0,
  18.0,
  44.0,
  50.0,
  68.0,
  30.0,
  11.0,
  70.0,
  16.0,
  15.0,
  58.0,
  57.0,
  57.0,
  37.0,
  14.0,
  11.0,
  23.0,
  7.0,
  32.0,
  36.0,
  7.0,
  34.0,
  48.0,
  43.0,
  11.0,
  16.0,
  30.0,
  67.0,
  15.0,
  42.0,
  14.0,
  24.0,
  14.0,
  35.0,
  8.0,
  20.0,
  7.0,
  10.0,
  26.0,
  22.0,
  7.0,
  41.0,
  27.0],
 [145.0,
  125.0,
  176.0,
  104.0,
  138.0,
  149.0,
  129.0,
  174.0,
  169.0,
  95.0,
  153.0,
  90.0,
  39.0,
  89.0,
  186.0,
  71.0,
  116.0,
  184.0,
  102.0,
  41.0,
  57.0,
  152.0,
  128.0,
  147.0,
  83.0,
  65.0,
  100.0,
  75.0,
  57.0,
  164.0,
  70.0,
  15.0,
  38.0,
  118.0,
  8.0,
  91.0,
  124.0,
  17.0,
  55.0,
  90.0,
  110.0,
  95.0,
  14.0,
  7.0,
  14.0,
  40.0,
  44.0,
  7.0,
  133.0,
  7.0],
 [56.0,
  69.0,
  157.0,
  91.0,
  132.0,
  91.0,
  61.0,
  27.0,
  86.0,
  75.0,
  62.0,
  90.0,
  94.0,
  38.0,
  132.0,
  40.0,
  88.0,
  71.0,
  57.0,
  41.0,
  64.0,
  14.0,
  84

In [38]:
lr = LogisticRegression(penalty='l1',C=0.5)
lr.fit(X_train_leaves, y_train)
y_pred_gbdtlr1 = lr.predict(X_test_leaves)
gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)

gbdt+lr auc 1: 0.34600


In [33]:
X_train_leaves=gbclf.apply(X_train)[:,:,0]
X_test_leaves=gbclf.apply(X_test)[:,:,0]

In [34]:
print(len(X_train_leaves))
print(len(X_train_leaves[0]))
y_pred_gbdtlr1 = lr.predict(X_test_leaves)
gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)

5794
40
gbdt+lr auc 1: 0.34814


In [52]:
gbc=GradientBoostingClassifier()
gbc.fit(X_train,y_train)
y_pred=gbc.predict_proba(X_test)[:,1]
auc=roc_auc_score(y_test,y_pred)
print(auc)

0.930927618648


In [46]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
y_pred_xgb=xgb.predict_proba(X_test)[:,1]
auc=roc_auc_score(y_test,y_pred_xgb)
print(auc)

0.931977598518


In [135]:
gnb= GaussianNB()
gnb.fit(X_train_leaves, y_train)
Y_pred_nb=gnb.predict_proba(X_test_leaves)[:,1]
gnb_auc = roc_auc_score(y_test,Y_pred_nb)
print('NB auc: ', gnb_auc)

NB auc:  0.657571872229


In [136]:
svc=SVC(probability=True)

svc.fit(X_train_leaves, y_train)
Y_pred_svc=svc.predict_proba(X_test_leaves)[:,1]
svc_auc=roc_auc_score(y_test,Y_pred_svc)
print('SVC auc: %.5f' % svc_auc)

SVC auc: 0.65157


In [137]:
knn=KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train_leaves, y_train)
Y_pred_knn=knn.predict_proba(X_test_leaves)[:,1]
knn_auc=roc_auc_score(y_test,Y_pred_knn)
print('KNN auc : %.5f' % knn_auc)

KNN auc : 0.80715


In [138]:
#perceptron
perceptron = Perceptron()
perceptron.fit(X_train_leaves, y_train)
Y_pred_prec=perceptron.predict(X_test_leaves)
perc_auc=roc_auc_score(y_test,Y_pred_prec)
print('Perceptron auc : ', perc_auc)

Perceptron auc :  0.633917089256


In [139]:
#linear svc
lin = LinearSVC()
lin.fit(X_train_leaves, y_train)
Y_pred_lin=lin.predict(X_test_leaves)
lin_auc=roc_auc_score(y_test,Y_pred_lin)
print('Linear SVC auc : ', lin_auc)

Linear SVC auc :  0.51237250333


In [140]:
#SGD
sgd = SGDClassifier(loss='log')
sgd.fit(X_train_leaves, y_train)
Y_pred_sgd=sgd.predict_proba(X_test_leaves)[:, 1]
sgd_auc=roc_auc_score(y_test,Y_pred_sgd)
print('SGD auc : %.5f' % sgd_auc)

SGD auc : 0.73946


In [141]:
#XGB

xgb=XGBClassifier()
xgb.fit(X_train_leaves,y_train)
Y_pred_xgb=xgb.predict(X_test_leaves)
xgb_auc= roc_auc_score(y_test,Y_pred_xgb)
print(xgb_auc)

0.633002491391


In [142]:
lr = LogisticRegression(n_jobs=-1)
X_train_ext = hstack([X_train_leaves, X_train])
lr.fit(X_train_ext, y_train)
X_test_ext = hstack([X_test_leaves, X_test])
y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
gbdtlr_auc2 = roc_auc_score(y_test, y_pred_gbdtlr2)
print('gbdt+lr auc 2: %.5f' % gbdtlr_auc2)

gbdt+lr auc 2: 0.80278
